In [ ]:
import pandas as pd
from scipy import stats
import numpy as np
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

In [ ]:
df = pd.read_csv('/content/pamap2.csv').drop('Unnamed: 0', axis =1)
df

,timestamp,activityID,heartrate,handTemperature,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,...,ankleGyro2,ankleGyro3,ankleMagne1,ankleMagne2,ankleMagne3,ankleOrientation1,ankleOrientation2,ankleOrientation3,ankleOrientation4,subject_id
0,37.70,1,100.0,30.3750,2.30106,7.25857,6.09259,2.20720,7.24042,5.95555,...,-0.048878,-0.006328,-60.2040,-37.1225,-57.8847,1.000000,0.000000,0.000000,0.000000,1.0
1,37.81,1,100.0,30.3750,2.24615,7.48180,5.55219,2.25130,7.40595,5.65313,...,-0.017912,0.015243,-60.7820,-36.2116,-58.4891,1.000000,0.000000,0.000000,0.000000,1.0
2,37.92,1,100.0,30.3750,2.30000,7.10681,6.09309,2.34613,7.46611,6.12103,...,-0.011798,0.011387,-61.4531,-36.2983,-58.4845,1.000000,0.000000,0.000000,0.000000,1.0
3,38.03,1,100.0,30.3750,2.49455,7.52335,6.17157,2.59106,7.76653,6.27112,...,-0.009233,0.005823,-62.0065,-36.7217,-58.3608,1.000000,0.000000,0.000000,0.000000,1.0
4,38.14,1,101.0,30.3750,2.71654,8.30596,4.78671,2.59596,7.94641,4.86635,...,-0.042562,-0.009289,-60.4311,-37.5412,-59.0013,1.000000,0.000000,0.000000,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52485,2269.19,4,114.0,31.6875,-10.29120,3.39539,4.30053,-9.47897,3.10943,4.02856,...,-0.748676,5.848500,-35.3362,25.6327,26.2167,0.313338,0.737921,0.252730,0.541682,4.0
52486,2269.30,4,114.0,31.6875,-10.75430,1.13390,5.57131,-10.69570,1.64004,6.01122,...,-0.112150,-1.868560,-27.0431,36.5420,23.9639,0.429451,0.758563,0.193857,0.450082,4.0
52487,2269.41,4,114.0,31.7500,-13.33730,1.10276,4.07020,-13.41370,1.00859,4.21840,...,-0.783090,-2.063760,-34.9815,30.1048,20.6813,0.401287,0.709983,0.331969,0.474014,4.0
52488,2269.52,4,114.0,31.7500,-11.09730,4.64814,3.66843,-11.19960,4.66223,3.96651,...,-0.095796,-0.563901,-38.5088,23.3364,23.3715,0.331919,0.700193,0.394961,0.493523,4.0


In [ ]:
hands_col = []
chest_col = []
for i in df.columns:
  if 'hand' in i and 'Orientation' not in i and 'Temperature' not in i:
    hands_col.append(i)
  elif 'chest' in i and 'Orientation' not in i and 'Temperature' not in i:
    chest_col.append(i)
hands_col.append('activityID')
chest_col.append('activityID')

In [ ]:
len(hands_col), len(chest_col)

(13, 13)

In [ ]:
hand_df = df[hands_col].reset_index()
chest_df = df[chest_col].reset_index()
hand_df

,index,handAcc16_1,handAcc16_2,handAcc16_3,handAcc6_1,handAcc6_2,handAcc6_3,handGyro1,handGyro2,handGyro3,handMagne1,handMagne2,handMagne3,activityID
0,0,2.30106,7.25857,6.09259,2.20720,7.24042,5.95555,-0.069961,-0.018328,0.004582,9.15626,-67.1825,-20.0857,1
1,1,2.24615,7.48180,5.55219,2.25130,7.40595,5.65313,-0.431227,0.002685,-0.062964,9.14612,-67.3936,-20.5508,1
2,2,2.30000,7.10681,6.09309,2.34613,7.46611,6.12103,0.075692,-0.030792,0.005246,9.69163,-67.0898,-21.2481,1
3,3,2.49455,7.52335,6.17157,2.59106,7.76653,6.27112,-0.259058,-0.267895,-0.038533,9.58694,-67.0882,-20.8997,1
4,4,2.71654,8.30596,4.78671,2.59596,7.94641,4.86635,0.377115,-0.023688,-0.020670,8.59622,-67.1486,-20.1947,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52485,52485,-10.29120,3.39539,4.30053,-9.47897,3.10943,4.02856,1.613480,1.466470,-1.076120,42.98670,-20.2877,-22.3692,4
52486,52486,-10.75430,1.13390,5.57131,-10.69570,1.64004,6.01122,0.161723,0.552174,-0.766122,47.37250,-16.7956,-17.5437,4
52487,52487,-13.33730,1.10276,4.07020,-13.41370,1.00859,4.21840,1.363810,-0.259642,0.250320,47.11230,-17.1527,-15.5725,4
52488,52488,-11.09730,4.64814,3.66843,-11.19960,4.66223,3.96651,0.113903,-0.865223,0.887103,45.78870,-20.1846,-15.8139,4


In [ ]:
timestep = 40
step = 10

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:


def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)


X_train, y_train = create_dataset(hand_df.drop('activityID' , axis =1),  hand_df['activityID'], timestep,step)
X_test, y_test = create_dataset(chest_df.drop('activityID' , axis =1), chest_df['activityID'], timestep, step)

print(X_train.shape, y_train.shape)

(5245, 40, 13) (5245, 1)


In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
print("X_train.shape: ", X_train.shape)
print("X_test.shape ", X_test.shape)
print("y_train.shape ", y_train.shape)
print("y_test.shape ", y_test.shape)

X_train.shape:  (5245, 40, 13)
X_test.shape  (5245, 40, 13)
y_train.shape  (5245, 14)
y_test.shape  (5245, 14)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# cnn model vary kernel size
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, LSTM, Conv1D, MaxPooling1D,Bidirectional
from tensorflow.keras.layers import TimeDistributed, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam


In [ ]:
#Epoch 500

verbose,epochs,batch_size=1,10,32

n_timesteps,n_features,n_outputs=X_train.shape[1],X_train.shape[2],y_train.shape[1]

In [ ]:
n_timesteps, n_features, n_outputs

(40, 13, 14)

In [ ]:
verbose,epochs,batch_size=1,10,32

n_timesteps,n_features,n_outputs=X_train.shape[1],X_train.shape[2],y_train.shape[1]
n_timesteps, n_features, n_outputs


(40, 13, 14)

In [ ]:
#CNN MODEL

inputs=keras.Input(shape=(n_timesteps,n_features))

conv_1=tf.keras.layers.Conv1D(filters=64,kernel_size=5,strides=2,activation='relu')(inputs)
maxpool_1=tf.keras.layers.MaxPooling1D(pool_size=2,strides=2)(conv_1)

conv_2=tf.keras.layers.Conv1D(filters=96,kernel_size=3,strides=1,activation='relu')(maxpool_1)
avg_pooling=tf.keras.layers.GlobalAveragePooling1D()(conv_2)
batch_norm=tf.keras.layers.BatchNormalization()(avg_pooling)

output=tf.keras.layers.Dense(n_outputs,activation='softmax')(batch_norm)
model=tf.keras.Model(inputs=inputs,outputs=output)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 40, 13)]          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 18, 64)            4224      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 9, 64)             0         
 g1D)                                                            
                                                                 
 conv1d_3 (Conv1D)           (None, 7, 96)             18528     
                                                                 
 global_average_pooling1d_1  (None, 96)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 batch_normalization_1 (Bat  (None, 96)                384 

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs = 100, batch_size = 32, validation_split = 0.1, shuffle = True)

Epoch 1/100
148/148 [==============================] - 1s 4ms/step - loss: 2.4024 - accuracy: 0.1695 - val_loss: 10.1120 - val_accuracy: 0.0000e+00
Epoch 2/100
148/148 [==============================] - 0s 3ms/step - loss: 2.0510 - accuracy: 0.1962 - val_loss: 3.7808 - val_accuracy: 0.0000e+00
Epoch 3/100
148/148 [==============================] - 0s 3ms/step - loss: 2.0012 - accuracy: 0.1932 - val_loss: 2.8972 - val_accuracy: 0.0000e+00
Epoch 4/100
148/148 [==============================] - 0s 3ms/step - loss: 1.9820 - accuracy: 0.2042 - val_loss: 7.4356 - val_accuracy: 0.0000e+00
Epoch 5/100
148/148 [==============================] - 0s 3ms/step - loss: 1.9769 - accuracy: 0.2051 - val_loss: 2.5222 - val_accuracy: 0.0000e+00
Epoch 6/100
148/148 [==============================] - 0s 3ms/step - loss: 2.0013 - accuracy: 0.1972 - val_loss: 2.5962 - val_accuracy: 0.1314
Epoch 7/100
148/148 [==============================] - 0s 3ms/step - loss: 1.9901 - accuracy: 0.1989 - val_loss: 3.8504 -

In [ ]:
base_loss,base_accuracy=model.evaluate(X_test,y_test,batch_size=batch_size,verbose=1)

score=base_accuracy*100
print('Accuracy >{:f}'.format(score))
print('Base Loss >{:.2f}'.format(base_loss))

164/164 [==============================] - 0s 1ms/step - loss: 1.8059 - accuracy: 0.4524
Accuracy >45.243087
Base Loss >1.81


In [ ]:
base_loss,base_accuracy=model.evaluate(X_test,y_test,batch_size=batch_size,verbose=1)
cnn_file='/content/CNN_Model.h5'

tf.keras.models.save_model(model, cnn_file, include_optimizer=False)
print('model saved at ', cnn_file)
#score,keras_file=evaluate_model(trainX,trainy,testX,testy)
score=base_accuracy*100
print('Accuracy >{:f}'.format(score))
print('Base Loss >{:.2f}'.format(base_loss))

164/164 [==============================] - 0s 1ms/step - loss: 1.8059 - accuracy: 0.4524
model saved at  /content/CNN_Model.h5
Accuracy >45.243087
Base Loss >1.81


<ipython-input-53-fcb20a907118>:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, cnn_file, include_optimizer=False)


In [ ]:
base_model = keras.models.load_model(cnn_file)

In [ ]:
new_model = keras.models.Sequential(base_model.layers[:-1])
base_model.trainable = False
new_model.add(keras.layers.Flatten())
new_model.add(keras.layers.Dense(14, activation="sigmoid"))

In [ ]:
new_model.compile(loss="categorical_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

history2 = new_model.fit(X_test, y_test, epochs=30, batch_size = 32, validation_split = 0.1, shuffle = True)


Epoch 1/30
148/148 [==============================] - 1s 4ms/step - loss: 2.7711 - accuracy: 0.1110 - val_loss: 2.9931 - val_accuracy: 0.0000e+00
Epoch 2/30
148/148 [==============================] - 1s 5ms/step - loss: 2.5056 - accuracy: 0.2030 - val_loss: 2.8912 - val_accuracy: 0.0000e+00
Epoch 3/30
148/148 [==============================] - 0s 3ms/step - loss: 2.3156 - accuracy: 0.2525 - val_loss: 2.8090 - val_accuracy: 0.0000e+00
Epoch 4/30
148/148 [==============================] - 0s 2ms/step - loss: 2.1833 - accuracy: 0.2752 - val_loss: 2.7415 - val_accuracy: 0.0000e+00
Epoch 5/30
148/148 [==============================] - 0s 2ms/step - loss: 2.0882 - accuracy: 0.3163 - val_loss: 2.6854 - val_accuracy: 0.0000e+00
Epoch 6/30
148/148 [==============================] - 0s 2ms/step - loss: 2.0169 - accuracy: 0.3189 - val_loss: 2.6374 - val_accuracy: 0.0000e+00
Epoch 7/30
148/148 [==============================] - 0s 2ms/step - loss: 1.9606 - accuracy: 0.3405 - val_loss: 2.5963 - val

In [ ]:
base_loss,base_accuracy=new_model.evaluate(X_test,y_test,batch_size=batch_size,verbose=1)

score=base_accuracy*100
print('Accuracy >{:f}'.format(score))
print('Base Loss >{:.2f}'.format(base_loss))

164/164 [==============================] - 0s 1ms/step - loss: 1.6423 - accuracy: 0.4402
Accuracy >44.022879
Base Loss >1.64


In [ ]:
new_model = keras.models.Sequential(base_model.layers[:-1])
base_model.trainable = True
new_model.add(keras.layers.Flatten())
new_model.add(keras.layers.Dense(14, activation="sigmoid"))

In [ ]:
new_model.compile(loss="categorical_crossentropy",

                     optimizer=keras.optimizers.SGD(lr=1e-3),

                     metrics=["accuracy"])



history2 = new_model.fit(X_train, y_train, epochs=30)

Epoch 1/30
164/164 [==============================] - 1s 4ms/step - loss: 2.5075 - accuracy: 0.1138
Epoch 2/30
164/164 [==============================] - 1s 4ms/step - loss: 2.1696 - accuracy: 0.1613
Epoch 3/30
164/164 [==============================] - 1s 4ms/step - loss: 2.1213 - accuracy: 0.1750
Epoch 4/30
164/164 [==============================] - 0s 3ms/step - loss: 2.0781 - accuracy: 0.1619
Epoch 5/30
164/164 [==============================] - 0s 2ms/step - loss: 2.0421 - accuracy: 0.1798
Epoch 6/30
164/164 [==============================] - 0s 2ms/step - loss: 2.0357 - accuracy: 0.1746
Epoch 7/30
164/164 [==============================] - 0s 3ms/step - loss: 2.0202 - accuracy: 0.1806
Epoch 8/30
164/164 [==============================] - 0s 3ms/step - loss: 2.0011 - accuracy: 0.1880
Epoch 9/30
164/164 [==============================] - 0s 2ms/step - loss: 2.0428 - accuracy: 0.1706
Epoch 10/30
164/164 [==============================] - 0s 2ms/step - loss: 2.0104 - accuracy: 0.1729

In [ ]:
base_loss,base_accuracy=new_model.evaluate(X_test,y_test,batch_size=batch_size,verbose=1)

score=base_accuracy*100
print('Accuracy >{:f}'.format(score))
print('Base Loss >{:.2f}'.format(base_loss))

164/164 [==============================] - 0s 1ms/step - loss: 2.0462 - accuracy: 0.2334
Accuracy >23.336510
Base Loss >2.05
